## Consumer Script for Loading Data from Kafka in MongoDB

In [1]:
from kafka import KafkaAdminClient, KafkaConsumer
from kafka.admin import NewTopic
import pymongo
import threading

In [2]:
admin = KafkaAdminClient(bootstrap_servers="localhost:9092")

if "preise"not in admin.list_topics() and "stationen" not in admin.list_topics():
    newTopics = list()
    newTopics.append(NewTopic(name="preise", num_partitions=1, replication_factor=1))
    newTopics.append(NewTopic(name="stationen", num_partitions=1, replication_factor=1))
    admin.create_topics(newTopics)

preisConsumer = KafkaConsumer(bootstrap_servers="localhost:9092", enable_auto_commit=False)
preisConsumer.subscribe("preise")

tankeConsumer = KafkaConsumer(bootstrap_servers="localhost:9092", enable_auto_commit=False)
tankeConsumer.subscribe("stationen")

client = pymongo.MongoClient('localhost', 27017)
db = client.tanken
preise = db.preise
stationen = db.stationen

In [3]:
def listenToConsumer(consumer, collection):
    print("started ",collection.name," listener.")
    for msg in consumer:
        item = msg.value.decode("utf-8")
        print(collection.name, " msg: ", item)
        collection.insert_one(item)
        consumer.commit_async()

In [4]:
threading.Thread(target=listenToConsumer, args=(preisConsumer, preise)).start()
threading.Thread(target=listenToConsumer, args=(tankeConsumer, stationen)).start()

started  preise  listener.
started  stationen  listener.
